In [1]:
!pip install transformers datasets torch gpustat wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/

In [2]:
# This is just form wandb to monitor the resoure states; you don't have to use it
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: che19028 (che19028-brigham-young-university-idaho) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
from datasets import Dataset
from tqdm.auto import tqdm
from google.colab import files
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments, pipeline
import json
import shutil
import pandas as pd
import torch
import os
import gpustat
import time
import re
import torch.multiprocessing as mp

In [4]:
uploaded = files.upload()

Saving raw.json to raw.json


Below is specifically for my data cleaning process.

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [56]:
file_name = list(uploaded.keys())[0]
file_path = f"/content/{file_name}"
df = pd.read_json(file_path)

def format_t5_data(row):
    input_text = row['input']
    output_dict = row['output']
    output_text = ", ".join([f"{key}: {value}" for key, value in output_dict.items()])
    return input_text, output_text

df_to_transform = df.drop(columns=['input', 'output'])

# Turn input and output columns into string
df_to_transform[['input_for_t5', 'output_for_t5']] = df.apply(format_t5_data, axis=1, result_type='expand')

# Save to a text file (tab-separated)
with open("/content/t5_data.txt", "w") as f:
  for row in df_to_transform.itertuples(index=False):
    f.write(f"{row.input_for_t5}\t{row.output_for_t5}\n")

def tokenize_data(example):
    input_encoding = tokenizer(example['input_for_t5'], return_tensors="pt", max_length=128, truncation=True, padding="max_length")
    output_encoding = tokenizer(example['output_for_t5'], return_tensors="pt", max_length=128, truncation=True, padding="max_length")
    return {"input_ids": input_encoding["input_ids"].squeeze(), "attention_mask": input_encoding["attention_mask"].squeeze(), "labels": output_encoding["input_ids"].squeeze()}

dataset = Dataset.from_pandas(df_to_transform) # create the dataset from the pandas dataframe.

tokenized_dataset = dataset.map(tokenize_data) # apply tokenization.

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [57]:
# Example: Print the first sample
print(dataset)
print(tokenized_dataset[0])

Dataset({
    features: ['input_for_t5', 'output_for_t5'],
    num_rows: 300
})
{'input_for_t5': 'Set up a conference call with the vendors this Monday at 2 PM.', 'output_for_t5': 'title: Vendor Conference Call, reminder: Prepare agenda, location: blank, date: 03/24/2025, time: 02:00 PM', 'input_ids': [2821, 95, 3, 9, 2542, 580, 28, 8, 8535, 48, 2089, 44, 204, 3246, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [58]:
# Split the dataset
train_dataset = tokenized_dataset.train_test_split(test_size=0.1)["train"]
train_dataset.set_format(type="torch")

val_dataset = tokenized_dataset.train_test_split(test_size=0.1)["test"]
val_dataset.set_format(type="torch")


In [59]:
print(train_dataset[0])

{'input_for_t5': 'Book a jewelry making workshop for this Monday evening at the craft store.', 'output_for_t5': 'title: Jewelry Workshop, reminder: blank, location: craft store, date: 03/24/2025, time: whole day', 'input_ids': tensor([3086,    3,    9, 8064,  492, 4786,   21,   48, 2089, 2272,   44,    8,
        5449, 1078,    5,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    

In [60]:
# Print device of a sample from the train dataset
print(f"Device of train_input_ids: {train_dataset[0]['input_ids'].device}")
print(f"Device of val_input_ids: {val_dataset[0]['input_ids'].device}")
print(f"Device of model: {model.device}")
print(f"CUDA is available: {torch.cuda.is_available()}")

if model.device.type != 'cuda':
    print("Model is not on GPU. Moving model to GPU.")
    model.to('cuda')
    print(f"Device of model: {model.device}")

Device of train_input_ids: cpu
Device of val_input_ids: cpu
Device of model: cuda:0
CUDA is available: True


A custom trainer if you don't want the trainer to pin the memory during the training proces.

In [ ]:
class CustomTrainer(Trainer):
    def _get_train_dataloader(self):
        # Modify the DataLoader for training
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.per_device_train_batch_size,
            shuffle=True,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=False  # Disable pin_memory for GPU datasets
        )

    def _get_eval_dataloader(self):
        # Modify the DataLoader for evaluation
        return DataLoader(
            self.eval_dataset,
            batch_size=self.args.per_device_eval_batch_size,
            shuffle=False,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=False  # Disable pin_memory for GPU datasets
        )

Define traing arguments and a trainer.

In [65]:
checkpoint_dir = "/content/tuned_t5"

mp.set_start_method('spawn', force=True)

# Training arguments
training_args = TrainingArguments(
    output_dir=checkpoint_dir,                        # output directory
    num_train_epochs=10,                              # number of training epochs
    per_device_train_batch_size=8,                   # batch size for training
    per_device_eval_batch_size=8,                   # batch size for evaluation
    warmup_steps=150,                                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                                # strength of weight decay
    logging_dir="/content/logs",                             # directory for storing logs
    logging_steps=10,                                 # log every X updates
    eval_strategy="steps",                            # evaluation strategy (here after every epoch)
    eval_steps=5,
    save_strategy="steps",                            # save model after each epoch
    save_steps=10,
    load_best_model_at_end=True,                      # load the best model at the end of training
    fp16=True,
    dataloader_num_workers=4,                           # increase if your CPU can handle it
    gradient_accumulation_steps=2,                       # gradient accumulation if batch size is small
    disable_tqdm=False,
    label_names=["labels"]
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator, # add the data collator
)

Where the magic begins.

In [ ]:
# Find the latest checkpoint safely
last_checkpoint = None
if os.path.exists(checkpoint_dir) and os.listdir(checkpoint_dir):
    checkpoints = [os.path.join(checkpoint_dir, d) for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint")]
    if checkpoints:
        last_checkpoint = max(checkpoints, key=os.path.getctime)  # Get latest checkpoint

# Check if resuming or starting fresh
if last_checkpoint:
    print(f"Resuming from checkpoint: {last_checkpoint}")
else:
    print("No checkpoint found. Starting training from scratch.")

print("Starting training...")
try:
    trainer.train(resume_from_checkpoint=last_checkpoint if last_checkpoint else None)
except Exception as e:
    print(f"Error during training: {e}")
    # Optionally, handle the error or exit gracefully

try:
    results = trainer.evaluate()
    print(f"Evaluation Results: {results}")
except Exception as e:
    print(f"Error during evaluation: {e}")

try:
    trainer.save_model(checkpoint_dir)
    tokenizer.save_pretrained(checkpoint_dir)
    print("Training completed successfully!")
except Exception as e:
    print(f"Error during saving: {e}")

No checkpoint found. Starting training from scratch.
Starting training...


Step,Training Loss,Validation Loss
5,No log,1.312191
10,2.786900,1.304529
15,2.786900,1.294595
20,2.803600,1.278946
25,2.803600,1.260822
30,2.615000,1.245596
35,2.615000,1.227900
40,2.389300,1.213473
45,2.389300,1.194023
50,2.323800,1.175340


In [ ]:
shutil.make_archive("tuned_t5", "zip", "/content/tuned_t5")  # Compress folder into ZIP
files.download("tuned_t5.zip")

This is for deleting the unwanted directory

In [66]:
# import shutil

# # Specify the path to the folder you want to delete
# folder_path = "/content/tuned_t5"

# # Delete the folder and all of its contents
# shutil.rmtree(folder_path)